## Qdrant をローカルで開始する方法
### ダウンロードして実行する


In [5]:
!docker pull qdrant/qdrant

1821.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using default tag: latest
latest: Pulling from qdrant/qdrant

8d6c47ec: Pulling fs layer 
d67c2f2b: Pulling fs layer 
6ae37626: Pulling fs layer 
b700ef54: Pulling fs layer 
446440f1: Pulling fs layer 
1c881c4f: Pulling fs layer 
6b2ae603: Pulling fs layer 
2cf4eb3c: Pull complete 057MB/2.057MBBDigest: sha256:7a4788934788a7ed9cbf6b8cc3ca1ee880dcd969cf8c6639dc7d0e446cbd4b47
Status: Downloaded newer image for qdrant/qdrant:latest
docker.io/qdrant/qdrant:latest


### 次に、サービスを実行します（-dオプションをつけてコンテナをバックグラウンドで動作させます）

In [ ]:
!docker run -d -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    qdrant/qdrant

### Qdrantのクライアントを初期化する

In [2]:
!pip3 install qdrant_client
from qdrant_client import QdrantClient


Defaulting to user installation because normal site-packages is not writeable


In [7]:
client = QdrantClient(url="http://localhost:6333")

### コレクションを追加する

In [ ]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)


### ベクトルの追加

In [8]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=2 status=<UpdateStatus.COMPLETED: 'completed'>


### クエリの実行

In [9]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)

[ScoredPoint(id=4, version=2, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=2, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=2, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


### フィルターを追加する

In [10]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=2, version=2, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
